## H2O modo local

#### Inicialização

In [40]:
import h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_65"; Java(TM) SE Runtime Environment (build 1.8.0_65-b17); Java HotSpot(TM) 64-Bit Server VM (build 25.65-b01, mixed mode)
  Starting server from /anaconda/envs/h2o-aula/h2o_jar/h2o.jar
  Ice root: /var/folders/41/lfsj80cj15vf7pvf3cn4tfjc0000gn/T/tmp8pw_gqug
  JVM stdout: /var/folders/41/lfsj80cj15vf7pvf3cn4tfjc0000gn/T/tmp8pw_gqug/h2o_cristoferweber_started_from_python.out
  JVM stderr: /var/folders/41/lfsj80cj15vf7pvf3cn4tfjc0000gn/T/tmp8pw_gqug/h2o_cristoferweber_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Sao_Paulo
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,4 months and 2 days !!!
H2O cluster name:,H2O_from_python_cristoferweber_fwvytj
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


#### Carga dos dados de arquivo CSV

Este arquivo é uma amostra do do dataset original em formato Parquet.

A amostra foi de 5% do conteúdo original.

Após amostragem, os seguintes filtros foram aplicados:

- Distância (trip_distance) >= 1
- Meio de pagamento Cartão (payment_type = 1)
- 'Taxa Gorjeta' entre 0 e 0.99

In [41]:
h2o_nyc_data = h2o.import_file("./data/nyc_yellow_taxi_sample.csv.gz",
                               destination_frame="nyc_data",
                               header=1,
                               sep=",")

Parse progress: |█████████████████████████████████████████████████████████| 100%


##### Colunas disponíveis no H2OFrame:

In [42]:
h2o_nyc_data.col_names

['tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pulocationid',
 'dolocationid',
 'ratecodeid',
 'fare_amount',
 'extra',
 'mta_tax',
 'improvement_surcharge',
 'Tip_amount',
 'Tolls_amount',
 'Total_amount']

##### Dimensões do H2OFrame:

In [43]:
h2o_nyc_data.dim

[4487239, 14]

#### Criação de algumas features discutidas em aula

Todas as features abaixo utilizam operadores e funções disponíveis no H2O.

A conversão de 'dayOfWeek' para factor utiliza a abreviatura do dia da semana.

In [44]:
h2o_nyc_data['tip_rate'] = h2o_nyc_data['Tip_amount'] / h2o_nyc_data['Total_amount']

h2o_nyc_data['pickup_hour'] = h2o_nyc_data['tpep_pickup_datetime'].hour()

h2o_nyc_data['pickup_weekday'] = h2o_nyc_data['tpep_pickup_datetime'].dayOfWeek().asfactor()

#### Um resumo do formato e conteúdo das variáveis.

**Vejam nos mínimos e máximos se existem valores que podem afetar o desempenho do modelo.**

In [ ]:
h2o_nyc_data.describe()

Rows:4487239
Cols:17




,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pulocationid,dolocationid,ratecodeid,fare_amount,extra,mta_tax,improvement_surcharge,Tip_amount,Tolls_amount,Total_amount,tip_rate,pickup_hour,pickup_weekday
type,time,time,int,real,int,int,int,real,real,real,real,real,real,real,real,int,enum
mins,1230809501000.0,1230811121000.0,0.0,1.0,1.0,1.0,1.0,0.33,-10.6,0.0,0.0,0.0,0.0,0.33,0.0,0.0,
mean,1498508399688.337,1498509575197.6294,1.6091090757590576,3.8096985206270504,161.80084323567365,159.14128732612633,1.0456057722800087,15.671919784081028,0.370041069798156,0.4978416683399289,0.29999104126167836,3.1381111168805567,0.49643400095247914,20.480890794985633,0.15209084854915264,13.750892252451921,
maxs,1512061372000.0,1512146986000.0,192.0,183.7,265.0,265.0,99.0,465.5,10.0,11.4,0.3,110.0,95.76,552.57,0.49358341559723595,23.0,
sigma,9238355312.782312,9238409669.085747,1.2616635046544722,4.1146427999242965,65.59491392448841,71.37036450617535,0.2849347571494506,11.965825624516713,0.4875973754721276,0.0333168296742776,0.0016393723912689214,2.746178963080608,1.7949487333486154,15.563014992495331,0.05009760293882924,6.485928844692992,
zeros,0,0,5711,0,0,0,0,0,2144685,19474,134,177766,4126412,0,177766,167070,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,2017-11-25 10:50:58,2017-11-25 11:06:01,1.0,3.8,68.0,209.0,1.0,14.5,0.0,0.5,0.3,3.05,0.0,18.35,0.1662125340599455,10.0,Sat
1,2017-11-25 10:37:41,2017-11-25 10:48:01,1.0,1.28,162.0,163.0,1.0,8.5,0.0,0.5,0.3,1.86,0.0,11.16,0.16666666666666669,10.0,Sat
2,2017-11-25 10:56:28,2017-11-25 11:04:18,1.0,1.41,163.0,43.0,1.0,7.5,0.0,0.5,0.3,1.0,0.0,9.3,0.1075268817204301,10.0,Sat


#### Inspeção das variáveis

Acesse o H2O Flow pelo comando abaixo, visualize o H2OFrame 'nyc_data' e analise as distribuições das variáveis que compõe o dataset. Pelas distribuições, você identifica outros valores que podem interferir na qualidade do modelo? Pense em valores extremos e valores pouco freqüentes. E na nossa variável dependente, 'tip_rate', você considera que valores fora das faixas comuns são facilmente capturáveis por um modelo?

In [ ]:
h2o.flow()

#### Treino e avaliação de modelos de regressão com H2O

**Divisão em Treino, Teste e Validação.**

Se o treino for muito lento, sugiro a redução do dataset de Treino para .6 e Validação para 0.15. Qual deve ser o tamanho do dataset de Teste?

In [70]:
train, test, valid = h2o_nyc_data.split_frame(ratios=[.7, .15], seed=42)

#### Regressão Linear com GLM

Detalhes de algoritmo e parâmetros discutidos em aula: [link para documentação](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/glm.html)

In [71]:
from h2o.estimators.glm import *

taxi_regression = H2OGeneralizedLinearEstimator(family="gaussian", model_id="taxi_glm_regression",
                                                alpha=[0.01, 0.05, 0.5, 0.95], lambda_search=True, nlambdas=75, 
                                                remove_collinear_columns=True)

taxi_regression.train(y="tip_rate", 
                      x=['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 
                         'Tolls_amount', 'pickup_hour', 'pickup_weekday'], 
                      training_frame=train,
                      validation_frame=valid)


glm Model Build progress: |

█

█

█

█

...

█

█

█

█

██████████████████████████████| 100%

In [52]:
taxi_regression

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1531074008389_2


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.002491489452113867
RMSE: 0.0499148219681676
MAE: 0.035698968857003584
RMSLE: 0.044530635335644456
R^2: 0.006929162609987216
Mean Residual Deviance: 0.002491489452113867
Null degrees of freedom: 3141205
Residual degrees of freedom: 3141192
Null deviance: 7880.889581261709
Residual deviance: 7826.281615916792
AIC: -9916749.512201494

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.002490594456154316
RMSE: 0.04990585593048491
MAE: 0.03567580147099961


In [54]:
taxi_regression.predict(test_data=test)

glm prediction progress: |████████████████████████████████████████████████| 100%


predict
0.153072
0.151512
0.147303
0.152401
0.150631
0.147521
0.152404
0.153974
0.151436
0.14893


In [56]:
from h2o.estimators.gbm import *

taxi_gbm_regression = H2OGradientBoostingEstimator(distribution="gaussian", model_id="taxi_gbm_regression",
                                                   ntrees=200, stopping_metric="rmse", stopping_rounds=7, stopping_tolerance=0.0005,
                                                   learn_rate=0.2, learn_rate_annealing=0.95,
                                                   max_depth=5, min_rows=2, col_sample_rate=0.95, sample_rate=0.7)  


In [57]:
taxi_gbm_regression.train(y="tip_rate", 
                          x=['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 
                             'Tolls_amount', 'pickup_hour', 'pickup_weekday'], 
                          training_frame=train,
                          validation_frame=valid)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [60]:
taxi_gbm_regression._model_json['output']

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,200.0,200.0,27914.0,0.0,5.0,0.8,1.0,32.0,5.83


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-07-08 15:46:48,0.011 sec,0.0,0.0500887,0.0358627,0.0025089,0.0500633,0.0358321,0.0025063
,2018-07-08 15:47:05,17.509 sec,13.0,0.0498058,0.0356100,0.0024806,0.0497987,0.0355891,0.0024799
,2018-07-08 15:47:20,32.137 sec,26.0,0.0497859,0.0355753,0.0024786,0.0497830,0.0355564,0.0024783
,2018-07-08 15:47:36,48.630 sec,62.0,0.0497829,0.0355700,0.0024783,0.0497810,0.0355516,0.0024781
,2018-07-08 15:47:57,1 min 9.519 sec,131.0,0.0497829,0.0355700,0.0024783,0.0497810,0.0355516,0.0024781
,2018-07-08 15:48:18,1 min 30.357 sec,200.0,0.0497829,0.0355700,0.0024783,0.0497810,0.0355516,0.0024781



ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.0024783366117992174
RMSE: 0.0497828947711884
MAE: 0.03556997501721137
RMSLE: 0.044414568140339145
Mean Residual Deviance: 0.0024783366117992174

ModelMetricsRegression: gbm
** Reported on validation data. **


In [67]:
from h2o.grid.grid_search import H2OGridSearch

gbm_params = {'learn_rate': [0.005, 0.01, 0.05, 0.1],
              'max_depth': [3, 5, 8],
              'sample_rate': [0.7, 0.85, 1.0],
              'min_rows': [3, 5, 8, 13, 21],
              'col_sample_rate': [0.7, 0.8, 0.95, 1.0]}

# COMBINAÇÕES ALEATÓRIAS DURANTE 25 MINUTOS ATÉ UM MÁXIMO DE 15 MODELOS
gbm_strategy = {'strategy': "RandomDiscrete", 'max_models': 15, 'max_runtime_secs': 25 * 60}

gbm_grid = H2OGridSearch(model=H2OGradientBoostingEstimator,
                         grid_id='gbm_grid_taxi',
                         hyper_params=gbm_params,
                         search_criteria=gbm_strategy)

gbm_grid.train(y="tip_rate", 
               x=['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 
                  'Tolls_amount', 'pickup_hour', 'pickup_weekday'], 
               training_frame=train, validation_frame=valid,
               distribution="gaussian",  
               ntrees=200, stopping_metric="rmse", stopping_rounds=7, stopping_tolerance=0.0005, seed=42)


gbm Grid Build progress: |██████████████████████████████████████████████

In [73]:
gbm_grid_performance = gbm_grid.get_grid(sort_by='rmse', decreasing=False)
gbm_grid_performance

Hyper-Parameter Search Summary:

ordered by increasing rmse

,col_sample_rate,learn_rate,max_depth,min_rows,sample_rate,model_ids,rmse
,0.8,0.1,8,3.0,1.0,gbm_grid_taxi_model_11,0.0497679
,0.8,0.1,8,3.0,0.7,gbm_grid_taxi_model_10,0.0497682
,1.0,0.05,5,21.0,1.0,gbm_grid_taxi_model_13,0.0497791
,0.95,0.1,5,5.0,0.85,gbm_grid_taxi_model_5,0.0497800
,0.8,0.1,5,3.0,1.0,gbm_grid_taxi_model_4,0.0497805
,0.8,0.05,3,13.0,0.7,gbm_grid_taxi_model_15,0.0497939
,0.7,0.1,3,3.0,1.0,gbm_grid_taxi_model_9,0.0497944
,0.8,0.1,3,21.0,0.85,gbm_grid_taxi_model_1,0.0497945
,0.8,0.1,3,13.0,1.0,gbm_grid_taxi_model_2,0.0497959
,0.8,0.1,3,21.0,1.0,gbm_grid_taxi_model_14,0.0497961


In [66]:
best_gbm_taxi = gbm_grid_performance.models[0]

gbm_taxi_perf = best_gbm_taxi.model_performance(test)

gbm_taxi_perf.rmse()

0.05012623343311321

In [39]:
h2o.cluster().shutdown(prompt=False)

H2O session _sid_9e37 closed.
